In [5]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
# Hàm giảm bộ nhớ (dùng cho pandas trước khi chuyển sang CuPy)
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# Đường dẫn tới file dữ liệu M5
SALES_PATH = 'C:/Users/Ho Hau/Downloads/M5/data/raw/sales_train_validation.csv'
CALENDAR_PATH = 'C:/Users/Ho Hau/Downloads/M5/data/raw/calendar.csv'
PRICES_PATH = 'C:/Users/Ho Hau/Downloads/M5/data/raw/sell_prices.csv'
OVERVIEW_DB_PATH = 'overview_data.db'

In [4]:
# Đọc một mẫu nhỏ của sales_df để lấy danh sách cột
sample_sales_df = pd.read_csv(SALES_PATH, nrows=1)
day_cols = [col for col in sample_sales_df.columns if col.startswith('d')]
sample_sales_df = None  # Giải phóng bộ nhớ

In [5]:
# Đọc dữ liệu chung và giảm bộ nhớ
calendar_df = pd.read_csv(CALENDAR_PATH)
prices_df = pd.read_csv(PRICES_PATH)
calendar_df = reduce_mem_usage(calendar_df)
prices_df = reduce_mem_usage(prices_df)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [6]:
# Kết nối SQLite
conn = sqlite3.connect(OVERVIEW_DB_PATH)
c = conn.cursor()

In [7]:
# Tạo bảng sales_raw
c.execute('''
    CREATE TABLE IF NOT EXISTS sales_raw (
        id TEXT,
        item_id TEXT,
        dept_id TEXT,
        cat_id TEXT,
        store_id TEXT,
        state_id TEXT,
        d TEXT,
        sales REAL,
        date TEXT,
        wm_yr_wk INTEGER,
        sell_price REAL
    )
''')

# Tạo bảng calendar và prices
calendar_df[['d', 'date', 'wm_yr_wk']].to_sql('calendar', conn, if_exists='replace', index=False)
prices_df.to_sql('prices', conn, if_exists='replace', index=False)

6841121

In [8]:
# Lấy danh sách các khu vực (store_id) duy nhất
sales_df = pd.read_csv(SALES_PATH, usecols=['store_id'])
unique_store_ids = sales_df['store_id'].unique()
sales_df = None  # Giải phóng bộ nhớ

In [9]:
# Xử lý từng khu vực
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
for store_id in unique_store_ids:
    print(f"Xử lý khu vực: {store_id}")
    
    # Đọc lại dữ liệu cho khu vực cụ thể (chỉ lấy hàng có store_id này)
    chunk = pd.read_csv(SALES_PATH, usecols=id_cols + day_cols)
    chunk = chunk[chunk['store_id'] == store_id]
    chunk = reduce_mem_usage(chunk)
    
    # Melt dữ liệu cho khu vực này
    sales_melted_chunk = chunk.melt(id_vars=id_cols, var_name='d', value_name='sales')
    sales_melted_chunk = reduce_mem_usage(sales_melted_chunk)
    
    # Gộp với calendar_df
    sales_with_date_chunk = sales_melted_chunk.merge(calendar_df[['d', 'date', 'wm_yr_wk']], on='d', how='left')
    sales_with_date_chunk = reduce_mem_usage(sales_with_date_chunk)
    
    # Gộp với prices_df
    sales_final_chunk = sales_with_date_chunk.merge(prices_df[['item_id', 'store_id', 'wm_yr_wk', 'sell_price']], 
                                                   on=['item_id', 'store_id', 'wm_yr_wk'], 
                                                   how='left')
    sales_final_chunk = reduce_mem_usage(sales_final_chunk)
    
    # Lưu vào SQLite
    sales_final_chunk.to_sql('sales_raw', conn, if_exists='append', index=False)
    print(f"Đã lưu dữ liệu cho {store_id} ({len(sales_final_chunk)} hàng)")
    
    # Giải phóng bộ nhớ
    del chunk, sales_melted_chunk, sales_with_date_chunk, sales_final_chunk
    import gc
    gc.collect()

Xử lý khu vực: CA_1
Mem. usage decreased to  6.52 Mb (85.4% reduction)
Mem. usage decreased to 322.63 Mb (0.0% reduction)
Mem. usage decreased to 378.25 Mb (0.0% reduction)
Mem. usage decreased to 389.38 Mb (0.0% reduction)
Đã lưu dữ liệu cho CA_1 (5832737 hàng)
Xử lý khu vực: CA_2
Mem. usage decreased to  5.82 Mb (87.0% reduction)
Mem. usage decreased to 322.63 Mb (0.0% reduction)
Mem. usage decreased to 378.25 Mb (0.0% reduction)
Mem. usage decreased to 389.38 Mb (0.0% reduction)
Đã lưu dữ liệu cho CA_2 (5832737 hàng)
Xử lý khu vực: CA_3
Mem. usage decreased to  8.89 Mb (80.1% reduction)
Mem. usage decreased to 322.63 Mb (0.0% reduction)
Mem. usage decreased to 378.25 Mb (0.0% reduction)
Mem. usage decreased to 389.38 Mb (0.0% reduction)
Đã lưu dữ liệu cho CA_3 (5832737 hàng)
Xử lý khu vực: CA_4
Mem. usage decreased to  5.74 Mb (87.1% reduction)
Mem. usage decreased to 322.63 Mb (0.0% reduction)
Mem. usage decreased to 378.25 Mb (0.0% reduction)
Mem. usage decreased to 389.38 Mb (0.0

In [10]:
# Lưu và đóng kết nối
conn.commit()
conn.close()

print("Đã tạo và điền dữ liệu thô vào overview_data.db theo từng khu vực!")

Đã tạo và điền dữ liệu thô vào overview_data.db theo từng khu vực!


In [4]:
OVERVIEW_DB_PATH = 'overview_data.db'
# Kết nối đến cơ sở dữ liệu
conn = sqlite3.connect(OVERVIEW_DB_PATH)
c = conn.cursor()

# Kiểm tra các bảng có trong cơ sở dữ liệu
print("Danh sách các bảng:")
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
for table in tables:
    print(f"- {table[0]}")

# Kiểm tra số lượng hàng trong từng bảng
for table in ['sales_raw', 'calendar', 'prices']:
    c.execute(f"SELECT COUNT(*) FROM {table}")
    count = c.fetchone()[0]
    print(f"Số lượng hàng trong bảng {table}: {count}")

# Kiểm tra dữ liệu mẫu từ từng bảng
print("\nDữ liệu mẫu từ bảng sales_raw (5 hàng đầu):")
c.execute("SELECT * FROM sales_raw LIMIT 5")
rows = c.fetchall()
for row in rows:
    print(row)

print("\nDữ liệu mẫu từ bảng calendar (5 hàng đầu):")
c.execute("SELECT * FROM calendar LIMIT 5")
rows = c.fetchall()
for row in rows:
    print(row)

print("\nDữ liệu mẫu từ bảng prices (5 hàng đầu):")
c.execute("SELECT * FROM prices LIMIT 5")
rows = c.fetchall()
for row in rows:
    print(row)

# Đóng kết nối
conn.close()

Danh sách các bảng:
- sales_raw
- calendar
- prices
Số lượng hàng trong bảng sales_raw: 58327370
Số lượng hàng trong bảng calendar: 1969
Số lượng hàng trong bảng prices: 6841121

Dữ liệu mẫu từ bảng sales_raw (5 hàng đầu):
('HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_001', 'HOBBIES_1', 'HOBBIES', 'CA_1', 'CA', 'd_1', 0.0, '2011-01-29', 11101, None)
('HOBBIES_1_002_CA_1_validation', 'HOBBIES_1_002', 'HOBBIES_1', 'HOBBIES', 'CA_1', 'CA', 'd_1', 0.0, '2011-01-29', 11101, None)
('HOBBIES_1_003_CA_1_validation', 'HOBBIES_1_003', 'HOBBIES_1', 'HOBBIES', 'CA_1', 'CA', 'd_1', 0.0, '2011-01-29', 11101, None)
('HOBBIES_1_004_CA_1_validation', 'HOBBIES_1_004', 'HOBBIES_1', 'HOBBIES', 'CA_1', 'CA', 'd_1', 0.0, '2011-01-29', 11101, None)
('HOBBIES_1_005_CA_1_validation', 'HOBBIES_1_005', 'HOBBIES_1', 'HOBBIES', 'CA_1', 'CA', 'd_1', 0.0, '2011-01-29', 11101, None)

Dữ liệu mẫu từ bảng calendar (5 hàng đầu):
('d_1', '2011-01-29', 11101)
('d_2', '2011-01-30', 11101)
('d_3', '2011-01-31', 11101)
('d_4'

In [6]:
import sqlite3
OVERVIEW_DB_PATH = 'overview_data.db'
# Kết nối đến cơ sở dữ liệu SQLite
conn = sqlite3.connect(OVERVIEW_DB_PATH)
c = conn.cursor()

# Hàm kiểm tra kiểu dữ liệu của các cột trong bảng
def check_column_types(table_name):
    print(f"Kiểu dữ liệu của các cột trong bảng {table_name}:")
    c.execute(f"PRAGMA table_info({table_name});")
    columns = c.fetchall()
    for column in columns:
        print(f"- {column[1]}: {column[2]}")  # column[1] là tên cột, column[2] là kiểu dữ liệu
    print()

# Kiểm tra kiểu dữ liệu của các bảng
for table in ['sales_raw', 'calendar', 'prices']:
    check_column_types(table)

# Đóng kết nối
conn.close()

Kiểu dữ liệu của các cột trong bảng sales_raw:
- id: TEXT
- item_id: TEXT
- dept_id: TEXT
- cat_id: TEXT
- store_id: TEXT
- state_id: TEXT
- d: TEXT
- sales: REAL
- date: TEXT
- wm_yr_wk: INTEGER
- sell_price: REAL

Kiểu dữ liệu của các cột trong bảng calendar:
- d: TEXT
- date: TEXT
- wm_yr_wk: INTEGER

Kiểu dữ liệu của các cột trong bảng prices:
- store_id: TEXT
- item_id: TEXT
- wm_yr_wk: INTEGER
- sell_price: REAL

